In [ ]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)
from sklearn.model_selection import train_test_split
from sklearn.impute import SimpleImputer
from sklearn.compose import make_column_transformer

from sklearn.model_selection import cross_val_score
from sklearn.preprocessing import OneHotEncoder
from sklearn.pipeline import make_pipeline

from sklearn.preprocessing import PolynomialFeatures   # For Polynomial Regression

from sklearn.linear_model import Ridge
from sklearn.linear_model import LinearRegression, LogisticRegression

from sklearn.ensemble import RandomForestRegressor

from sklearn.metrics import mean_squared_error

from sklearn.preprocessing import scale
from sklearn.model_selection import train_test_split

from sklearn.model_selection import GridSearchCV

from sklearn.compose import ColumnTransformer
from sklearn.pipeline import Pipeline
from sklearn.impute import SimpleImputer
from sklearn.preprocessing import StandardScaler, OrdinalEncoder, OneHotEncoder
from sklearn.decomposition import PCA
from sklearn.svm import SVR
from sklearn.svm import SVC
# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

In [ ]:
train_identity = pd.read_csv('/kaggle/input/ieee-fraud-detection/train_identity.csv')
train_transaction = pd.read_csv('/kaggle/input/ieee-fraud-detection/train_transaction.csv')
test_identity = pd.read_csv('/kaggle/input/ieee-fraud-detection/test_identity.csv')
test_transaction = pd.read_csv('/kaggle/input/ieee-fraud-detection/test_transaction.csv')
#review the shape
print(train_identity.shape)
print(train_transaction.shape)
print(test_identity.shape)
print(test_transaction.shape)

In [ ]:
id_train, id_test = [],[]
for i in range(1, 39):
    if i < 10:
        id_train.append('id_0'+str(i))
        id_test.append('id-0'+str(i))
    else:
        id_train.append('id_'+str(i))
        id_test.append('id-'+str(i))

v_list = []
for i in range(1, 340):
    v_list.append('V'+str(i))

In [ ]:
train_identity.drop(id_train, axis=1, inplace=True)
test_identity.drop(id_test, axis=1, inplace=True)
train_transaction.drop(v_list, axis=1, inplace=True)
test_transaction.drop(v_list, axis=1, inplace=True)

In [ ]:
#Join
train=train_transaction.join(train_identity,lsuffix='TransactionID',rsuffix='TransactionID')
test=test_transaction.join(test_identity,lsuffix='TransactionID',rsuffix='TransactionID')

In [ ]:
# train.drop('TransactionID',axis=1)
# test.drop('TransactionID',axis=1)
train.rename(columns={'TransactionIDTransactionID':'TransactionID'},inplace=True)
test.rename(columns={'TransactionIDTransactionID':'TransactionID'},inplace=True)

In [ ]:
train_float = train.select_dtypes(np.float64).columns.to_list()
test_float = test.select_dtypes(np.float64).columns.to_list()

In [ ]:
test[test_float] = test[test_float].fillna(0).astype('int16')
train[train_float] = train[train_float].fillna(0).astype('int16')

In [ ]:
# train=train.sample(15000)

In [ ]:
train.head()

In [ ]:
test.head()

In [ ]:
# train.drop(train_null, axis=1, inplace=True) 
# test.drop(test_null, axis=1, inplace=True) 

In [ ]:
X = train.drop('isFraud',axis=1)
y = train['isFraud']
x_train, x_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)
X_Real = test #test for kaggle

In [ ]:
names_numeric=x_train.describe().columns
names_categorical=x_train.describe(exclude='number').columns

In [ ]:
x_train

In [ ]:
 #3.Data Preprocessing 
## Pipeline auxiliary function work with the different data types
nominal = names_categorical
ordinal = []
numerical = names_numeric
#Ordinal Encoder
ordinal_p = Pipeline([
    ("imputer", SimpleImputer(strategy='most_frequent')),
    ("encoder", OrdinalEncoder()),
])
#Nominal Encoder
nominal_p = Pipeline([
    ("imputer", SimpleImputer(strategy='most_frequent')),
    ("encoder", OneHotEncoder(handle_unknown='ignore'))
])
#Numerical Scaling
numerical_p = Pipeline([
    ("imputer" , SimpleImputer(strategy='mean')), #median is the default strategy we can change to mean
    ("scaler", StandardScaler())
])

#Final Pipeline
preprocessing_pipeline = ColumnTransformer([
    ("nominal_prep", nominal_p, nominal),
    ("ordinal_prep", ordinal_p, ordinal),
    ("numerical_prep", numerical_p, numerical)
])
complete_pipeline = Pipeline([('preprocessor',preprocessing_pipeline)])
complete_pipeline.fit(x_train, y_train)

In [ ]:
# define the base models
level0 = list()
level0.append(('lr', LogisticRegression()))
level0.append(('knn', KNeighborsClassifier()))
level0.append(('cart', DecisionTreeClassifier()))
level0.append(('svm', SVC()))
level0.append(('bayes', GaussianNB()))
# define meta learner model
level1 = LogisticRegression()
# define the stacking ensemble
model = StackingClassifier(estimators=level0, final_estimator=level1, cv=5)
# fit the model on all available data
model.fit(x_train, y_test)
# make a prediction for one example
yhat = model.predict(x_test)
model.score(x_test,y_test)